In [1]:
import optuna
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm
from lightgbm import log_evaluation, early_stopping

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_squared_log_error



/Users/idris/miniconda3/envs/forecast/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def calculate_lags(df, col, lags, shift=0):
    for l in lags:
        df[f"lag_{col}_{l}"] = df.groupby(["ts_id"])[col].shift(shift + l).astype(np.float16)

def calculate_rollings(df, col, rollings, shift=0):
    for r in rollings:
        df[f"rol_mean_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).mean()
        df[f"rol_std_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).std()
        df[f"rol_min_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).min()
        df[f"rol_max_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).max()


def RMSLE(y_true:np.ndarray, y_pred:np.ndarray) -> np.float64:

    return np.sqrt(mean_squared_log_error(y_true, y_pred))

        
def CVGroupTimeSeriesSplit(X, prediction_length=16, step_forecast='days'):

    if step_forecast=='days':
        unit_time = 365
    elif step_forecast=='weeks':
        unit_time = 52
    elif step_forecast is None:
        raise ValueError("The 'step_forecast' parameter should not be None")
        
    list_idx_fold = [1, 2, 3, int((unit_time)/prediction_length), int((unit_time*2)/prediction_length)]
    
    #X['time_idx'] = ((X['date'] - X['date'].min()).dt.days).astype(int)
    XX = X.set_index('time_idx', drop=True)

    for idx_fold in list_idx_fold:
        
        print('idx_fold :', idx_fold)
        time_min_data, time_max_data = XX.index.min(), XX.index.max()
        mask_sample_max_train = time_max_data - (idx_fold * prediction_length)
        mask_sample_max_test = mask_sample_max_train + (prediction_length)

        train = XX.iloc[:mask_sample_max_train].reset_index(drop=False)
        test = XX.iloc[mask_sample_max_train:mask_sample_max_test].reset_index(drop=False)
                
        yield train.time_idx.unique(), test.time_idx.unique()

In [3]:
path = '/Users/idris/Documents/ds_project/forecast_store_sales/data/'
#cutoff = '2017-07-31'
cutoff = '2017-08-16'

col_to_ignore = ["date", "is_future", "forecast_step", "ts_id"]

TARGET_COL = 'sales'

prediction_length = 16
lags_target=list(range(1, 16))
rollings_target=[2, 4, 8, 16]

In [4]:
df_cutoff = pd.read_csv(path + f'fe/cutoff/{cutoff}/final_frame.csv', sep=';').drop('transactions', axis=1)
df_cutoff['date'] = pd.to_datetime(df_cutoff['date'])
df_cutoff['is_future'] = df_cutoff['is_future'].fillna(False)
df_cutoff["time_idx"] = ((df_cutoff["date"] - df_cutoff["date"].min()).dt.days).astype(int)

df_cv = df_cutoff.loc[df_cutoff["is_future"] == False].reset_index(drop=True)

clean = pd.read_csv(path + f'trainclean.csv', sep=';')
clean['date'] = pd.to_datetime(clean['date'])

In [5]:
prediction_length = 16

lags_target=list(range(1, 16))
rollings_target=[2, 4, 8, 16]

time_val = 6

In [6]:
for train_idx, test_idx in tqdm(CVGroupTimeSeriesSplit(df_cv, prediction_length=16)):
        
        #i_train = df_cv.loc[train_idx, 'time_idx'].unique()
        df_train = df_cv.loc[df_cv['time_idx'].isin(train_idx)].drop(columns=col_to_ignore)
        #i_test = df_cv.loc[test_idx, 'time_idx'].unique()[:16]
        df_test = df_cv.loc[df_cv['time_idx'].isin(test_idx)].drop(columns=col_to_ignore).drop(columns=TARGET_COL) 

        min_train_idx = df_test['time_idx'].min() - prediction_length
        max_train_idx = df_test['time_idx'].max()
        cutoff_idx = np.arange(min_train_idx, max_train_idx + 1, 1)
        df_rec = df_cv.loc[df_cv['time_idx'].isin(cutoff_idx)]    
        
        break

0it [00:00, ?it/s]

idx_fold : 1


0it [00:03, ?it/s]


In [7]:
def objective(trial):
     
    params = {
        "objective": "tweedie",
        #"tweedie_variance_power": trial.suggest_float("tweedie_variance_power", 1.1, 1.6, step=0.1),
        "tweedie_variance_power": trial.suggest_categorical("tweedie_variance_power", [1.2, 1.3, 1.4]),
        "n_estimators": trial.suggest_categorical("n_estimators", [2000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
        "random_state": 1990,
        "metric": "rmse",
        "verbose": -1
    }
    
    list_rmsle = []
    
    for train_idx, test_idx in tqdm(CVGroupTimeSeriesSplit(df_cv, prediction_length=16)):
    
        df_train = df_cv.loc[df_cv['time_idx'].isin(train_idx)].drop(columns=col_to_ignore)
        df_test = df_cv.loc[df_cv['time_idx'].isin(test_idx)].drop(columns=col_to_ignore).drop(columns=TARGET_COL) 

        min_train_idx = df_test['time_idx'].min() - prediction_length
        max_train_idx = df_test['time_idx'].max()
        cutoff_idx = np.arange(min_train_idx, max_train_idx + 1, 1)
        df_rec = df_cv.loc[df_cv['time_idx'].isin(cutoff_idx)]    

        df_rec.loc[df_rec['time_idx'].isin(test_idx), 'is_future'] = True
        df_rec.loc[df_rec["is_future"] == True, "sales"] = np.nan
        
        i_val = df_train.time_idx.max() - time_val

        df_train_val = df_train.loc[df_train['time_idx'] <= i_val]
        df_val = df_train.loc[df_train['time_idx'] > i_val]
        
        print('train id limit', i_val)
        print('val id', df_val.time_idx.unique())
        print('id time to pred', test_idx)
        day, month, year = df_val.iloc[-1]['day'], df_val.iloc[-1]['month'], df_val.iloc[-1]['year']
        print(f'date: {day}/{month}/{year}')
        
        train_ds = lgb.Dataset(
        data=df_train_val.drop(columns=TARGET_COL), 
        label=df_train_val[TARGET_COL]
        )
        
        val_ds = lgb.Dataset(
        data=df_val.drop(columns=TARGET_COL), 
        label=df_val[TARGET_COL]
        )

        gbm = lgb.train(
        params=params,
        train_set=train_ds,
        valid_sets=val_ds,
        callbacks=[log_evaluation(period=50), early_stopping(100)]
        )

        first_future_time_idx = test_idx.min()
        df_rec.loc[:, "forecast_step"] = df_rec.loc[:, "time_idx"] - first_future_time_idx + 1 

        for fs in range(1, prediction_length + 1):

            # predict one step
            df_predict = df_rec.loc[df_rec["forecast_step"] == fs].drop(columns=col_to_ignore + [TARGET_COL])
            predictions = np.clip(gbm.predict(df_predict), a_min=0, a_max=None)

            # update sales_quantity usings predictions
            df_rec.loc[df_rec["forecast_step"] == fs, "sales"] = predictions

            # update target lags & rollings
            calculate_lags(df_rec, TARGET_COL, lags_target)
            calculate_rollings(df_rec, TARGET_COL, rollings_target)

        df_forecast = df_rec.loc[df_rec["is_future"] == True, ["ts_id", "date", "sales"]]
        df_forecast["forecast"] = df_forecast["sales"].round(0).astype(np.float16)
        df_forecast.drop(columns="sales", inplace=True)

        df_error = pd.merge(df_forecast, clean[['ts_id', 'date', 'sales']], how="left", on=['ts_id', 'date'])
        rmsle = RMSLE(df_error["sales"], df_error["forecast"])
        print('rmsle: ', rmsle)
        list_rmsle.append(rmsle)
            
    rmsle_mean = np.mean(list_rmsle)
    
    return rmsle_mean

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=25)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-27 14:07:23,306] A new study created in memory with name: no-name-0282ae77-d3ab-4666-9a70-f40380f6b518
0it [00:00, ?it/s]

idx_fold : 1
train id limit 1665
val id [1666 1667 1668 1669 1670 1671]
id time to pred [1672 1673 1674 1675 1676 1677 1678 1679 1680 1681 1682 1683 1684 1685
 1686 1687]
date: 30/7/2017
Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 236.459
[100]	valid_0's rmse: 223.98
[150]	valid_0's rmse: 221.412
[200]	valid_0's rmse: 220.846
Early stopping, best iteration is:
[117]	valid_0's rmse: 218.21


1it [01:28, 88.31s/it]

rmsle:  0.46895950988968627
idx_fold : 2
train id limit 1649
val id [1650 1651 1652 1653 1654 1655]
id time to pred [1656 1657 1658 1659 1660 1661 1662 1663 1664 1665 1666 1667 1668 1669
 1670 1671]
date: 14/7/2017
Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 233.729
[100]	valid_0's rmse: 216.856
[150]	valid_0's rmse: 210.471
[200]	valid_0's rmse: 206.927
[250]	valid_0's rmse: 206.556
[300]	valid_0's rmse: 205.586
[350]	valid_0's rmse: 205.672
[400]	valid_0's rmse: 204.492
[450]	valid_0's rmse: 204.465
[500]	valid_0's rmse: 204.388
[550]	valid_0's rmse: 205.178
Early stopping, best iteration is:
[476]	valid_0's rmse: 203.953


2it [03:45, 117.16s/it]

...

rmsle:  0.4685412360408719
idx_fold : 3
train id limit 1633
val id [1634 1635 1636 1637 1638 1639]
id time to pred [1640 1641 1642 1643 1644 1645 1646 1647 1648 1649 1650 1651 1652 1653
 1654 1655]
date: 28/6/2017
Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 275.616
[100]	valid_0's rmse: 260.194
[150]	valid_0's rmse: 255.623
[200]	valid_0's rmse: 246.629
[250]	valid_0's rmse: 246.369
[300]	valid_0's rmse: 246.688
[350]	valid_0's rmse: 251.936
[400]	valid_0's rmse: 245.618
[450]	valid_0's rmse: 240.119
[500]	valid_0's rmse: 240.161
[550]	valid_0's rmse: 237.674
[600]	valid_0's rmse: 245.968
Early stopping, best iteration is:
[532]	valid_0's rmse: 236.337


3it [05:13, 106.16s/it]

rmsle:  0.47410268720163806
idx_fold : 22
train id limit 1329
val id [1330 1331 1332 1333 1334 1335]
id time to pred [1336 1337 1338 1339 1340 1341 1342 1343 1344 1345 1346 1347 1348 1349
 1350 1351]
date: 28/8/2016
Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 152.024
[100]	valid_0's rmse: 149.018
[150]	valid_0's rmse: 146.544
[200]	valid_0's rmse: 141.636
[250]	valid_0's rmse: 140.178
[300]	valid_0's rmse: 141.502
[350]	valid_0's rmse: 139.637
[400]	valid_0's rmse: 138.284
[450]	valid_0's rmse: 137.98
[500]	valid_0's rmse: 139.363
Early stopping, best iteration is:
[403]	valid_0's rmse: 137.716


4it [06:34, 96.26s/it] 

rmsle:  0.77714863812039
idx_fold : 45
train id limit 961
val id [962 963 964 965 966 967]
id time to pred [968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983]
date: 26/8/2015


In [ ]:
1

In [ ]:
print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
trial.report

In [ ]:
optuna.visualization.plot_optimization_history(trial, target_name="Accuracy")

In [ ]:
no-name-e8c5307e-bc31-4758-8944-a3a96836696d

In [ ]:
def objective(trial):
     
    params = {
        "objective": "tweedie",
        #"tweedie_variance_power": trial.suggest_float("tweedie_variance_power", 1.1, 1.6, step=0.1),
        "tweedie_variance_power": trial.suggest_categorical("tweedie_variance_power", [1.3, 1.4]),
        "n_estimators": trial.suggest_categorical("n_estimators", [2000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
        "random_state": 1990,
        "metric": "rmse",
        "verbose": -1
    }
    
    list_wape = []
    
    for train_idx, test_idx in tqdm(GroupTimeSeriesSplit(n_splits=8).split(df_cv, groups=df_cv['time_idx'])):
    
        i_train = df_cv.loc[train_idx, 'time_idx'].unique()
        df_train = df_cv.loc[df_cv['time_idx'].isin(i_train)].drop(columns=col_to_ignore)
        i_test = df_cv.loc[test_idx, 'time_idx'].unique()[:16]
        df_test = df_cv.loc[df_cv['time_idx'].isin(i_test)].drop(columns=col_to_ignore).drop(columns=TARGET_COL) 

        min_train_idx = df_test['time_idx'].min() - prediction_length
        max_train_idx = df_test['time_idx'].max()
        cutoff_idx = np.arange(min_train_idx, max_train_idx + 1, 1)
        df_rec = df_cv.loc[df_cv['time_idx'].isin(cutoff_idx)]    

        df_rec.loc[df_rec['time_idx'].isin(i_test), 'is_future'] = True
        df_rec.loc[df_rec["is_future"] == True, "sales"] = np.nan
        
        i_val = df_train.time_idx.max() - time_val

        df_train_val = df_train.loc[df_train['time_idx'] <= i_val]
        df_val = df_train.loc[df_train['time_idx'] > i_val]
        
        print('train id limit', i_val)
        print('val id', df_val.time_idx.unique())
        print('id time to pred', i_test)
        day, month, year = df_val.iloc[-1]['day'], df_val.iloc[-1]['month'], df_val.iloc[-1]['year']
        print(f'date: {day}/{month}/{year}')
        
        train_ds = lgb.Dataset(
        data=df_train_val.drop(columns=TARGET_COL), 
        label=df_train_val[TARGET_COL]
        )
        
        val_ds = lgb.Dataset(
        data=df_val.drop(columns=TARGET_COL), 
        label=df_val[TARGET_COL]
        )

        gbm = lgb.train(
        params=params,
        train_set=train_ds,
        valid_sets=val_ds,
        callbacks=[log_evaluation(period=50), early_stopping(100)]
        )

        first_future_time_idx = i_test.min()
        df_rec.loc[:, "forecast_step"] = df_rec.loc[:, "time_idx"] - first_future_time_idx + 1 

        for fs in range(1, prediction_length + 1):

            # predict one step
            df_predict = df_rec.loc[df_rec["forecast_step"] == fs].drop(columns=col_to_ignore + [TARGET_COL])
            predictions = np.clip(gbm.predict(df_predict), a_min=0, a_max=None)

            # update sales_quantity usings predictions
            df_rec.loc[df_rec["forecast_step"] == fs, "sales"] = predictions

            # update target lags & rollings
            calculate_lags(df_rec, TARGET_COL, lags_target)
            calculate_rollings(df_rec, TARGET_COL, rollings_target)

        df_forecast = df_rec.loc[df_rec["is_future"] == True, ["ts_id", "date", "sales"]]
        df_forecast["forecast"] = df_forecast["sales"].round(0).astype(np.float16)
        df_forecast.drop(columns="sales", inplace=True)

        df_error = pd.merge(df_forecast, clean[['ts_id', 'date', 'sales']], how="left", on=['ts_id', 'date'])
        wape = np.round(np.sum(np.abs(df_error["sales"] - df_error["forecast"])) / np.sum(df_error["sales"]), 3)
        print('wape: ', wape)
        list_wape.append(wape)
    
    mean_wape = np.mean(list_wape)
    
    return mean_wape

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
trial.number

In [ ]:
Value: 0.1476
  Params: 
    tweedie_variance_power: 1.3
    n_estimators: 2000
    learning_rate: 0.08064028714756555
    num_leaves: 2140
    max_depth: 12
    min_data_in_leaf: 200
    lambda_l1: 10
    lambda_l2: 55
    min_gain_to_split: 7.674179670757204
    bagging_fraction: 0.7
    bagging_freq: 1
    feature_fraction: 0.9

In [ ]:
 Trial 0 finished with value: 0.14940000000000003 and parameters: 

{
    'tweedie_variance_power': 1.4000000000000001,
    'n_estimators': 500,
    'learning_rate': 0.067307721852165,
    'num_leaves': 300,
    'max_depth': 10,
    'min_data_in_leaf': 1300,
    'lambda_l1': 100,
    'lambda_l2': 80,
    'min_gain_to_split': 9.72024574054834,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'feature_fraction': 0.6000000000000001
}

. Best is trial 0 with value: 0.14940000000000003.

In [ ]:
Trial 7 finished with value: 0.156 and parameters: {'tweedie_variance_power': 1.3, 'n_estimators': 500, 'learning_rate': 0.047505552909073004, 'num_leaves': 2780, 'max_depth': 9, 'min_data_in_leaf': 4400, 'lambda_l1': 65, 'lambda_l2': 40, 'min_gain_to_split': 9.020620157726453, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 7 with value: 0.156.

In [ ]:
Trial 0 finished with value: 0.15299999999999997 and parameters: {'tweedie_variance_power': 1.4000000000000001, 'n_estimators': 500, 'learning_rate': 0.06908883769088184, 'num_leaves': 600, 'max_depth': 10, 'min_data_in_leaf': 400, 'lambda_l1': 5, 'lambda_l2': 50, 'min_gain_to_split': 3.4052603539384463, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 0 with value: 0.15299999999999997.

In [ ]:
df_cutoff[df_cutoff['ts_id'] == '9_28'].sort_values('date', ascending=False).head(20)

In [ ]:
prediction_length = 8
lags_target=list(range(1, 16))
rollings_target=[2, 4, 8, 16]

In [ ]:
def objective(trial):

    def calculate_lags(df, col, lags, shift=0):
        for l in lags:
            df[f"lag_{col}_{l}"] = df.groupby(["ts_id"])[col].shift(shift + l).astype(np.float16)

    def calculate_rollings(df, col, rollings, shift=0):
        for r in rollings:
            df[f"rol_mean_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).mean()
            df[f"rol_std_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).std()
            df[f"rol_min_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).min()
            df[f"rol_max_{col}_{r}"] = df.groupby(["ts_id"])[col].shift(shift + 1).rolling(r, min_periods=1).max()


    params = {
        "boosting_type": "gbdt", 
        "num_leaves": trial.suggest_int("num_leaves", 4, 10),
        "max_depth": -1, 
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1), 
        "n_estimators": 500, #100
        "subsample_for_bin": 200000, 
        "objective": trial.suggest_categorical("objective", ["regression", "tweedie"]),
        "class_weight": None, 
        "min_split_gain": 0.0,
        "min_child_weight": 0.001, 
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100, step=10),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0, step=0.1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0, step=0.1),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1.0, step=0.1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1.0, step=0.1),
        "random_state": 666,
        "metric": "rmse"
    }
    
    params["num_leaves"] = 2**params["num_leaves"]-1
    
    if params["objective"] == "tweedie":
        params["tweedie_variance_power"] = trial.suggest_float("tweedie_variance_power", 1.0, 1.9, step=0.1)
    
    if params["subsample"] < 1.0:
        params["subsample_freq"] = trial.suggest_int("subsample_freq", 1, 7)
    


    train_ds = lgb.Dataset(
        data=df_train.drop(columns=TARGET_COL), 
        label=df_train[TARGET_COL]
    )

    print("Fit")
    gbm = lgb.train(
        params=params,
        train_set=train_ds,
        valid_sets=train_ds
        )

    df_cutoff.loc[df_cutoff["is_future"] == True, "sales"] = np.nan

    for fs in range(1, prediction_length + 1):
        print(fs)

        # predict one step
        df_predict = df_cutoff.loc[df_cutoff["forecast_step"] == fs].drop(columns=col_to_ignore + [TARGET_COL])
        predictions = np.clip(gbm.predict(df_predict), a_min=0, a_max=None)

        # update sales_quantity usings predictions
        df_cutoff.loc[df_cutoff["forecast_step"] == fs, "sales"] = predictions

        # update target lags & rollings
        calculate_lags(df_cutoff, TARGET_COL, lags_target)
        calculate_rollings(df_cutoff, TARGET_COL, rollings_target)
    
    print("Format")
    df_forecast = df_cutoff.loc[df_cutoff["is_future"] == True, ["ts_id", "date", "sales"]]
    df_forecast["forecast"] = df_forecast["sales"].round(0).astype(np.float16)
    df_forecast.drop(columns="sales", inplace=True)
    print("Log metrics")
    df_error = pd.merge(df_forecast, clean[['ts_id', 'date', 'sales']], how="left", on=['ts_id', 'date'])
    wape = np.round(np.sum(np.abs(df_error["sales"] - df_error["forecast"])) / np.sum(df_error["sales"]), 4)
    
    return wape

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
trial.datetime_complete